In [183]:
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

In [184]:
id = pd.MultiIndex.from_product([['fro', 2], [100, 300, 500]])
df = pd.DataFrame(index = id)

In [185]:
T = 300
# cov_str = 'Cai2011Adaptive_Model2_my'
cov_str = 'Cai2011Adaptive_Model1'

In [186]:
simu_str = 'lx_band'
data_path = 'data_2023-4-1/'
for N in [100, 300, 500]:
    S = gen_S_Cai2011Adaptive_Model1(N)
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for eta in [0.5, 0.8, 1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            
            data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [187]:
simu_str = 'LSRthreshold'
data_path = 'data_LSRthreshold_2023-4-5/'
for N in [100, 300, 500]:
    S = gen_S_Cai2011Adaptive_Model1(N)
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for tau in [0.2]:
            for prob in [0.99, 0.9, 1]:
                for qrob in [0, 0.01, 0.1]:
                    cov_params = [cov_str]
                    simu_params = [simu_str, tau, prob, qrob, 'brute']
                    file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                    file_name = data_path + file_name
                    
                    try:
                        data = np.loadtxt(file_name)
                    except FileNotFoundError:
                        continue

                    col = str(simu_params)
                    if col not in df.columns:
                        df[col] = None
                    
                    data *= normS # absolute error
                    df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [188]:
simu_str = '' # to be determined
data_path = 'data_OtherMethods_4-5/'

for N in [100, 300, 500]:
    S = gen_S_Cai2011Adaptive_Model1(N)
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']:
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [189]:
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       8.76(0.76)       6.54(0.31)     5.69(0.25)   
    300      15.23(0.61)      10.61(0.44)     8.01(0.24)   
    500      19.35(0.79)      13.24(0.58)     9.47(0.21)   
2   100       3.48(0.38)       2.57(0.30)     2.53(0.23)   
    300       5.00(0.41)       3.70(0.36)     3.18(0.27)   
    500       5.39(0.38)       3.72(0.28)     3.24(0.25)   

        ['LSRthreshold', 0.2, 0.99, 0.01, 'brute']  \
fro 100                                 9.01(0.64)   
    300                                24.09(1.18)   
    500                                38.02(2.39)   
2   100                                 2.67(0.28)   
    300                                 4.69(0.31)   
    500                                 6.29(0.42)   

        ['LSRthreshold', 0.2, 0.99, 0.1, 'brute']  \
fro 100                                9.66(0.50)   
    300                               26.23(1.10)   
    500                               41.95(1.57)   
2   100                                2.85(0.33)   
    300                                5.11(0.29)   
    500                                6.73(0.47)   

        ['LSRthreshold', 0.2, 0.9, 0.01, 'brute']  \
fro 100                                9.00(0.52)   
    300                               24.15(1.41)   
    500                               38.31(2.08)   
2   100                                2.64(0.28)   
    300                                4.70(0.26)   
    500                                6.30(0.43)   

        ['LSRthreshold', 0.2, 0.9, 0.1, 'brute']       Sample Soft Threshold  \
fro 100                               9.67(0.42)  14.57(0.33)     9.49(3.90)   
    300                              26.46(0.94)  43.55(0.37)   16.63(10.98)   
    500                              42.13(2.10)  72.36(0.43)    13.44(1.98)   
2   100                               2.83(0.27)   4.57(0.40)     3.16(1.07)   
    300                               5.08(0.28)   9.26(0.43)     4.06(2.13)   
    500                               6.76(0.31)  12.84(0.39)     3.06(0.44)   

        Hard Threshold Linear Shrink Nonlinear Shrink  
fro 100    12.93(3.09)   12.19(0.20)       7.49(0.30)  
    300    30.33(9.51)   29.02(0.11)             None  
    500    37.88(0.07)   41.34(0.09)             None  
2   100     4.02(1.07)    3.67(0.33)       3.50(0.39)  
    300     8.21(1.87)    5.59(0.16)             None  
    500     8.99(0.02)    6.27(0.11)             None

In [190]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Augmented Band ($\eta={eta}$)'
    elif 'LSRthreshold' in col:
        t, p, q = col[1:-1]
        new_col = fr'Augmented Threshold ($\tau={t}, p={p}, q={q}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols

In [191]:
buf = 'latex/' + cov_str + '.tex'
caption = cov_str

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'll|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = "my label", 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_19500\4238430270.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [192]:
pd.get_option('display.width')

80

In [ ]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
        # if not write_flag:
        #     if r'begin{tabular}' in line:
